In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itbr"
fs_method, fs_ratio = "mireg", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9112 | AUPR: 0.6417 | Acc: 95.32%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9163 | AUPR: 0.6573 | Acc: 95.40%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9217 | AUPR: 0.6718 | Acc: 95.35%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9133 | AUPR: 0.6445 | Acc: 95.24%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9185 | AUPR: 0.6618 | Acc: 95.44%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9220 | AUPR: 0.6751 | Acc: 95.48%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9220 | AUPR: 0.6751 | Acc: 95.48%
Internal test: AUC: 0.9246 | AUPR: 0.6955 | Acc: 95.26%
External test: AUC: 0.9204 | AUPR: 0.6686 | Acc: 95.22%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9143 | AUPR: 0.6282 | Acc: 95.06%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9168 | AUPR: 0.6355 | Acc: 95.10%
model(kernel=linear, C=1)
Valid:         AUC: 0.9158 | AUPR: 0.6293 | Acc: 95.02%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9168 | AUPR: 0.6355 | Acc: 95.10%
Internal test: AUC: 0.9205 | AUPR: 0.6732 | Acc: 94.77%
External test: AUC: 0.9156 | AUPR: 0.6135 | Acc: 94.61%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7539 | AUPR: 0.4132 | Acc: 94.65%
model(n_neighbors=5)
Valid:         AUC: 0.7754 | AUPR: 0.4583 | Acc: 94.94%
model(n_neighbors=7)
Valid:         AUC: 0.7922 | AUPR: 0.4825 | Acc: 94.95%
best model(n_neighbors=7)
Valid:         AUC: 0.7922 | AUPR: 0.4825 | Acc: 94.95%
Internal test: AUC: 0.8112 | AUPR: 0.5209 | Acc: 94.74%
External test: AUC: 0.7937 | AUPR: 0.4850 | Acc: 94.80%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8731 | AUPR: 0.4050 | Acc: 86.68%
best model()
Valid:         AUC: 0.8731 | AUPR: 0.4050 | Acc: 86.68%
Internal test: AUC: 0.8857 | AUPR: 0.4426 | Acc: 87.13%
External test: AUC: 0.8705 | AUPR: 0.3873 | Acc: 85.71%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9173 | AUPR: 0.6554 | Acc: 95.28%
model(C=10.0)
Valid:         AUC: 0.9170 | AUPR: 0.6551 | Acc: 95.35%
model(C=100.0)
Valid:         AUC: 0.9168 | AUPR: 0.6545 | Acc: 95.33%
best model(C=1.0)
Valid:         AUC: 0.9173 | AUPR: 0.6554 | Acc: 95.28%
Internal test: AUC: 0.9229 | AUPR: 0.6890 | Acc: 95.13%
External test: AUC: 0.9144 | AUPR: 0.6373 | Acc: 94.94%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8837 | AUPR: 0.5660 | Acc: 85.82%
model(criterion=log_loss)
Valid:         AUC: 0.8875 | AUPR: 0.5655 | Acc: 87.61%
model(criterion=entropy)
Valid:         AUC: 0.8875 | AUPR: 0.5655 | Acc: 87.61%
best model(criterion=log_loss)
Valid:         AUC: 0.8875 | AUPR: 0.5655 | Acc: 87.61%
Internal test: AUC: 0.8758 | AUPR: 0.5841 | Acc: 85.43%
External test: AUC: 0.8792 | AUPR: 0.5655 | Acc: 84.92%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9170 | AUPR: 0.6580 | Acc: 90.04%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9186 | AUPR: 0.6602 | Acc: 90.18%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9189 | AUPR: 0.6603 | Acc: 90.21%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9179 | AUPR: 0.6599 | Acc: 90.07%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9188 | AUPR: 0.6610 | Acc: 90.09%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9190 | AUPR: 0.6621 | Acc: 90.32%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9179 | AUPR: 0.6599 | Acc: 90.07%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9188 | AUPR: 0.6610 | Acc: 90.09%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9190 | AUPR: 0.6621 | Acc: 90.32%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9190 | AUPR: 0.6621 | Ac

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9211 | AUPR: 0.6432 | Acc: 95.19%
model(criterion=squared_error)
Valid:         AUC: 0.9213 | AUPR: 0.6457 | Acc: 95.16%
best model(criterion=squared_error)
Valid:         AUC: 0.9213 | AUPR: 0.6457 | Acc: 95.16%
Internal test: AUC: 0.9246 | AUPR: 0.6473 | Acc: 94.96%
External test: AUC: 0.9199 | AUPR: 0.6369 | Acc: 94.98%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9184 | AUPR: 0.6470 | Acc: 95.25%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9178 | AUPR: 0.6488 | Acc: 95.30%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9177 | AUPR: 0.6490 | Acc: 95.28%
best model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9184 | AUPR: 0.6470 | Acc: 95.25%
Internal test: AUC: 0.9226 | AUPR: 0.6847 | Acc: 95.28%
External test: AUC: 0.9130 | AUPR: 0.6339 | Acc: 95.05%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)